# PART 1                (INGEST THE DATA)

In [2]:
import pandas as pd
import streamlit as st
from datetime import datetime

# Ingest the data
def ingest_data(file_path):
    
    ##Ingest the data from the TSV file into a pandas DataFrame.
    
    chunks = pd.read_csv(file_path, sep='\t', low_memory=False, parse_dates=['ts1'], chunksize=10000)
    df=pd.concat(chunks, ignore_index=True)
    return df

df = ingest_data('correct_twitter_202102.tsv')

# Preview the data
df[['id', 'ts1', 'text', 'author_id', 'created_at', 'like_count', 'place_id']].head()

,id,ts1,text,author_id,created_at,like_count,place_id
0,1358322479136178177,2022-03-01 09:06:52.403595-05:00,RT @briebxrries: what is joe biden and kamala ...,54603468,2021-02-07 02:51:30-05:00,0,None
1,1358323016736796677,2022-01-05 08:34:04.477789-05:00,RT @Variety: Britney Spears Documentary Brings...,754687840290680833,2021-02-07 02:53:38-05:00,0,None
2,1358322996696465409,2022-03-01 09:06:52.399008-05:00,RT @Johnnerkell: Britney Spears documentary is...,3297058346,2021-02-07 02:53:33-05:00,0,None
3,1358322976769286151,2022-03-01 09:06:52.399205-05:00,RT @PopCrave: .@NYTimes documentary #FramingBr...,38561622,2021-02-07 02:53:28-05:00,0,None
4,1368923802260889606,2022-03-01 07:26:05.505986-05:00,Hey y’all! I’m new to Britney stan twitter! Fe...,1343250140522573824,2021-03-08 08:57:22-05:00,0,None


# PART 2 (QUERYING FUNCTIONALITY)

In [45]:
# Query 1: Daily Term Frequency

def tweets_per_day(df, term):
    
    df['ts1'] = pd.to_datetime(df['ts1'], errors='coerce')
    ##Return how many tweets were posted containing the term on each day.
    term_tweets = df[df['text'].str.contains(term, case=False, na=False)]
    daily_count = term_tweets.groupby(term_tweets['ts1'].dt.date).size()
    return daily_count

In [44]:
# Query 2: Unique Users Posting the Term

def unique_user_count(df, term):
    
    ##Return the number of unique users who posted a tweet containing the term.
    
    term_tweets = df[df['text'].str.contains(term, case=False, na=False)]
    unique_users = term_tweets['author_id'].nunique()
    return unique_users


In [43]:
# Query 3: Average Likes for Tweets Containing the Term
def get_average_likes(df, term):
    
    ## Ensure 'like_count' is numeric, converting any non-numeric values to NaN
    
    df['like_count'] = pd.to_numeric(df['like_count'], errors='coerce')
    
    ##Return the average number of retweets for tweets containing the term.
    
    term_tweets = df[df['text'].str.contains(term, case=False, na=False)]
    average_likes = term_tweets['like_count'].sum().mean()
    return average_likes


In [42]:
# Query 4: Locations (Place IDs) for Tweets
def get_place_ids(df, term):
    
    ##Return the unique place IDs for tweets containing the term.

    term_tweets = df[df['text'].str.contains(term, case=False, na=False)]
    place_ids = term_tweets['place_id'].dropna().unique()
    return place_ids

In [41]:
# Query 5: Tweet Posting Times
def get_tweet_times(df, term):
    
    ##Return the times of day (hours) when tweets containing the term were posted.
    
    term_tweets = df[df['text'].str.contains(term, case=False, na=False)].copy()
    term_tweets.loc[:, 'hour'] = term_tweets['ts1'].dt.hour
    hourly_distribution = term_tweets.groupby('hour').size()
    return hourly_distribution

In [39]:
# Query 6: Most Active User Posting the Term
def get_most_active_user(df, term):
    
    ##Return the user who posted the most tweets containing the term.
    
    term_tweets = df[df['text'].str.contains(term, case=False, na=False)]
    most_active_user = term_tweets.groupby('author_id').size().idxmax()
    return most_active_user


In [38]:
#Specify a search term (e.g., "britney")
search_term = 'britney'
    
# Run the queries
print("\nDaily tweet frequency:", tweets_per_day(df, search_term))
print("\nUnique users posting the term:", unique_user_count(df, search_term))
print("\nAverage likes for tweets containing the term:", get_average_likes(df, search_term))
print("\nPlace IDs for tweets containing the term:", get_place_ids(df, search_term))
print("\nTweet posting times (hourly):", get_tweet_times(df, search_term))
print("\nMost active user posting the term:", get_most_active_user(df, search_term))


Daily tweet frequency: ts1
2022-01-04      7808
2022-01-05     79774
2022-01-22      1012
2022-03-01    661857
dtype: int64

Unique users posting the term: 386648

Average likes for tweets containing the term: 6062590.0

Place IDs for tweets containing the term: ['None' '97bcdfca1a2dca59' '7ef79c5ab17d518c' ... '5d2456d6a8ee247d'
 'd6819fe60643ebc1' '268b1c6b7ddcaa1b']

Tweet posting times (hourly): hour
6      13632
7     248379
8     284484
9     195136
15        38
16       583
17      8199
dtype: int64

Most active user posting the term: 632414122
